# Rigid Body Simulator

Credit to Professor Bo Zhu, on whose COSC 89 Course Notes the following code is based on

## Geometry in space

UNFINISHED

Question: by adding a third dimension to all the vectors to allow for cross products, the direction of gravity must be flipped to align with our frame of reference (ie now gravity is (0, 2, 0). why is this?

In [43]:
import taichi as ti
import numpy as np
import math

ti.init()

# variables and fields to keep track of
num_particles = 1
dt = 1e-3
substeps = 10
mass = 1

I = 10 # Inertial tensor
x = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles) # position of center of mass
vertices_0 = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles*3) # vertices of triangles relative to COM
rotated_vertices = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles*3)
force = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles)
torque = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles)
v = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles) # velocity
theta = ti.field(dtype=ti.f32, shape=num_particles) # orientation
w = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles) # angular velocity

@ti.kernel
def init_pos():    
    x[0] = ti.Vector([0.5, 0.5, 0])
    # equilateral triangle
    vertices_0[0] = ti.Vector([0, 0.1, 0])
    vertices_0[1] = ti.Vector([-0.0866, -0.05, 0])
    vertices_0[2] = ti.Vector([0.0866, -0.05, 0])
    rotated_vertices[0] = ti.Vector([0, 0.1, 0])
    rotated_vertices[1] = ti.Vector([-0.0866, -0.05, 0])
    rotated_vertices[2] = ti.Vector([0.0866, -0.05, 0])
    ### this is assuming a one object system
    theta[0] = math.pi/2
    v[0] = ti.Vector([0, 0, 0]) # init velocity
    force[0] = ti.Vector([0,0,0]) # init force
    torque[0] = ti.Vector([0,0,0]) 
    w[0] = ti.Vector([0, 0, 1]) # small amount of angular to start
    
@ti.func
def clear_force():
    for i in range(num_particles):
        force[i] = ti.Vector([0.0, 0.0, 0.0])
    
@ti.func
def calculate_force():
    clear_force()
    # gravity
    for i in range(num_particles):
        force[i] = force[i] - ti.Vector([0, 0, 0])

@ti.func
def rotate():
    t = theta[0]
    R = ti.Vector([[ti.cos(t), -ti.sin(t), 0], [ti.sin(t), ti.cos(t), 0], [0, 0, 0]])
    vec = ti.Vector([vertices_0[0][0], vertices_0[0][1], vertices_0[0][2]])
    result = R @ vec
    rotated_vertices[0] = ti.Vector([result[0], result[1], result[2]])

@ti.func
def calculate_torque():
    # no torque from gravity since r = 0
    pass

@ti.kernel
def substep():
    calculate_force()
    calculate_torque()
    # update velocity
    v[0] = v[0] + dt * force[0] / mass
    # update angular velocity
    w[0] = w[0] + dt * torque[0] / I
    # update position
    x[0] = x[0] + dt * v[0]
    # update orientation
#     theta[0] = theta[0] * dt * w[0][2]
    # update vertices
    rotate()

def main():
    gui = ti.GUI('Rigid Body', (600, 600))
    # initialize vertices
    init_pos()
        
    while gui.running:
        for e in gui.get_events(ti.GUI.PRESS):
            pass
            
        # Update for a time step
        for step in range(substeps):
            substep()

        print(theta[0])
        gui.triangle(a=x[0]+rotated_vertices[0], b=x[0]+rotated_vertices[1], c=x[0]+rotated_vertices[2], color=0xED553B)
        
        gui.show()
    
if __name__ == '__main__':
    main()

[Taichi] Starting on arch=x64
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1.5707963705062866
1

Code from stackoverflow on finding the inertial tensor using triangles

https://stackoverflow.com/questions/41592034/computing-tensor-of-inertia-in-2d